In [1]:
from datasets import load_dataset

ds = load_dataset("cbasu/Med-EASi")

print(ds)
print("Train dataset sample: ")
ds['train'][0]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1397 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/196 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 1397
    })
    validation: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 196
    })
    test: Dataset({
        features: ['Expert', 'Simple', 'Annotation', 'sim', 'sentence_sim', 'compression', 'expert_fk_grade', 'expert_ari', 'layman_fk_grade', 'layman_ari', 'umls_expert', 'umls_layman', 'expert_terms', 'layman_terms', 'idx'],
        num_rows: 300
    })
})
Train dataset sample: 


{'Expert': '75-90 % of the affected people have mild intellectual disability.',
 'Simple': "People with syndromic intellectual disabilities may have a `` typical look. ''",
 'Annotation': "<del>75-90 % of the</del> <rep>affected people  have mild intellectual disability.<by>People with syndromic intellectual disabilities</rep> <ins>may have a `` typical look. ''</ins>",
 'sim': 0.48951049,
 'sentence_sim': 0.639872432,
 'compression': 1.2,
 'expert_fk_grade': 12.7,
 'expert_ari': 12.4,
 'layman_fk_grade': 13.1,
 'layman_ari': 15.1,
 'umls_expert': "[[{'start': 41, 'end': 64, 'ngram': 'intellectual disability', 'term': 'intellectual disability', 'cui': 'C3714756', 'similarity': 1.0, 'semtypes': {'T048'}, 'preferred': 1, 'preferred_term': None}, {'start': 41, 'end': 64, 'ngram': 'intellectual disability', 'term': 'Intellectual disability', 'cui': 'C3714756', 'similarity': 0.9090909090909091, 'semtypes': {'T048'}, 'preferred': 0, 'preferred_term': None}, {'start': 41, 'end': 64, 'ngram': 

In [2]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")


def preprocess_and_tokenizer(batch):
    inputs = batch['Expert']
    targets = batch['Simple']

    model_inputs = tokenizer(inputs, truncation=True, max_length=256)
    labels = tokenizer(text_target=targets, truncation=True, max_length=256)["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs


col_names = ds["train"].column_names
tokenized_ds = ds.map(preprocess_and_tokenizer, batched=True, remove_columns=col_names)

tokenized_ds

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1397 [00:00<?, ? examples/s]

Map:   0%|          | 0/196 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1397
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 196
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})

In [3]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments


model_name = "google/flan-t5-base"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints/flan_t5_baseline",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=3e-5,
    num_train_epochs=5,
    warmup_steps=1000,
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    load_best_model_at_end=True,
    tpu_num_cores=8,
    predict_with_generate=True,
    save_total_limit=2,
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


trainer.train()

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/tmp/ipython-input-2474634736.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vsonnaku (vsonnaku-george-mason-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
wandb.finish()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Try loading tokenizer from local checkpoint, fallback to original pretrained tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained("checkpoints/flan_t5_baseline")
except Exception:
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

model = AutoModelForSeq2SeqLM.from_pretrained("checkpoints/flan_t5_baseline")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# List of complex medical texts to simplify
examples = [
    "The patient was diagnosed with congestive heart failure and exhibited pulmonary oedema requiring diuretics and oxygen supplementation.",
    "Administration of 5-fluorouracil can lead to myelosuppression and mucositis; dose adjustments may be necessary in elderly patients.",
    "Hyponatraemia in this context is likely due to SIADH caused by the small-cell lung carcinoma, necessitating fluid restriction and close monitoring of sodium levels.",
    "The MRI demonstrates a hyperintense lesion on T2-weighted images consistent with demyelination in the periventricular white matter.",
    "The patient presents with chronic kidney disease stage 3b, eGFR 35 mL/min/1.73m2, and requires medication dose review to avoid nephrotoxicity."
]

instruction_template = (
    "Simplify the following medical text for a general audience without changing meaning:\n\n"
)

for i, text in enumerate(examples, 1):
    prompt = instruction_template + text
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=2
        )
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # Print original and simplified outputs
    print(f"Example {i} - Original:\n{text}\n")
    print(f"Example {i} - Simplified:\n{decoded[0]}\n")
    print("-" * 80)